In [ ]:
# 02 - Procesamiento y ETL con PySpark

# Este notebook replica el flujo de análisis y ETL usando PySpark y carga los resultados en `warehouse/warehouse_pyspark.db`. Ejecuta este notebook desde la raíz del proyecto.


# 1. Configuración Inicial

Se importa PySpark y se crea una SparkSession para procesar datos de manera distribuida. Se definen rutas relativas para el dataset y la base de datos SQLite.

In [24]:
import os

import pandas as pd
import sqlalchemy
from pyspark.sql import SparkSession, functions as F, Window

# Rutas relativas desde notebooks/ hacia data/ y warehouse/
DATA_PATH = "../data/videogames.csv"
DB_PATH = "../warehouse/warehouse_pyspark.db"
DB_URL = f"sqlite:///{DB_PATH}"

spark = SparkSession.builder.appName("videogames_pyspark_etl").getOrCreate()

print(f"CSV: {DATA_PATH}")
print(f"SQLite DB (PySpark): {DB_PATH}")


CSV: ../data/videogames.csv
SQLite DB (PySpark): ../warehouse/warehouse_pyspark.db


# 2. Carga del Dataset

Se carga el dataset de videojuegos usando Spark DataFrame con spark.read.csv, habilitando inferencia de esquema. Se muestra el esquema, dimensiones y primeras filas para verificar la carga.

In [25]:
# Carga del dataset con Spark

raw_df = (
    spark.read
    .option("header", True)  # Indica que el archivo CSV tiene una fila de encabezados
    .option("inferSchema", True)  # Le dice a Spark que detecte automáticamente el tipo de datos
    .csv(DATA_PATH)
)

print("===== Dataset cargado =====")
raw_df.printSchema()
print(f"Dimensiones del dataset: {raw_df.count()} filas x {len(raw_df.columns)} columnas")
print("\nPrimeras 5 filas:")
raw_df.show(5)


===== Dataset cargado =====
root
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- cost: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- pegi: string (nullable = true)
 |-- year: string (nullable = true)
 |-- developer: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- region: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- engine: string (nullable = true)
 |-- award: string (nullable = true)
 |-- dlc_support: string (nullable = true)
 |-- language: string (nullable = true)
 |-- metascore: string (nullable = true)
 |-- user_score: string (nullable = true)
 |-- reviews: string (nullable = true)
 |-- rating_source: string (nullable = true)
 |-- copies_sold_millions: string (nullable = true)
 |-- revenue_millions_usd: string (nullable = true)

Dimensiones del dataset: 10000 filas x 21 columnas

Primeras 5 filas:
+-------------------+-------+-----+--------+----

# 3. Limpieza de Datos

Se eliminan duplicados con dropDuplicates y se tratan nulos: imputación con media para numéricas y "Unknown" para categóricas. Se convierten formatos especiales en columnas numéricas.

In [26]:
# Tratamiento de nulos y duplicados en Spark

print("===== 1. Eliminación de duplicados =====")
initial_count = raw_df.count()
clean_df = raw_df.dropDuplicates()
final_count = clean_df.count()
print(f"Filas antes: {initial_count}, después: {final_count}")

# Identificamos columnas numéricas y categóricas
# Nota: Como inferSchema=True detecta todo como string, no habrá columnas numéricas aquí
# Las convertiremos después en la siguiente celda
numeric_cols = [f.name for f in clean_df.schema.fields if str(f.dataType) in ("IntegerType", "LongType", "DoubleType", "FloatType")] 
cat_cols = [f.name for f in clean_df.schema.fields if f.name not in numeric_cols]

print(f"\nColumnas numéricas detectadas: {numeric_cols}")
print(f"Columnas categóricas detectadas: {len(cat_cols)} columnas")

# Tratamiento de nulos en columnas numéricas (si las hay)
if numeric_cols:
    for col in numeric_cols:
        mean_val = clean_df.select(F.mean(F.col(col))).first()[0]
        if mean_val is not None:
            clean_df = clean_df.na.fill({col: float(mean_val)})
            print(f"Columna '{col}' rellenada con media: {mean_val}")
        else:
            clean_df = clean_df.na.fill({col: 0.0})
            print(f"Columna '{col}' rellenada con 0.0 (sin valores válidos)")

# Tratamiento de nulos en columnas categóricas
# También tratamos valores problemáticos como "?", "N/A", etc.
print("\n===== 2. Tratamiento de valores faltantes en columnas categóricas =====")
for col in cat_cols:
    # Reemplazar valores problemáticos comunes con "Unknown"
    clean_df = clean_df.withColumn(
        col,
        F.when(
            (F.col(col).isNull()) | 
            (F.col(col) == "?") | 
            (F.col(col) == "N/A") |
            (F.col(col) == ""),
            "Unknown"
        ).otherwise(F.col(col))
    )

print("Valores faltantes y problemáticos reemplazados con 'Unknown'")

print("\n===== Dataset después de limpieza inicial =====")
clean_df.show(5)


===== 1. Eliminación de duplicados =====


Filas antes: 10000, después: 10000

Columnas numéricas detectadas: []
Columnas categóricas detectadas: 21 columnas

===== 2. Tratamiento de valores faltantes en columnas categóricas =====
Valores faltantes y problemáticos reemplazados con 'Unknown'

===== Dataset después de limpieza inicial =====
+--------------------+----------+-------+--------+------------------+-------+----+----------+-----------+------+-----------+-------------+---------+-----------+--------+---------+----------+-----------------+-------------+--------------------+--------------------+
|                name|     genre|   cost|platform|        popularity|   pegi|year| developer|  publisher|region|       mode|       engine|    award|dlc_support|language|metascore|user_score|          reviews|rating_source|copies_sold_millions|revenue_millions_usd|
+--------------------+----------+-------+--------+------------------+-------+----+----------+-----------+------+-----------+-------------+---------+-----------+--------+---

# 4. Normalización de Datos

Se normalizan nombres de columnas a snake_case y se convierten tipos de datos. Se limpian valores problemáticos en categóricas y se preparan los datos para el modelo dimensional.

In [27]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType

print("\n===== 2. Limpieza de columnas de ventas/ingresos =====")

numeric_target_cols = ["copies_sold_millions", "revenue_millions_usd"]

for col in numeric_target_cols:
    if col in clean_df.columns:

        # 1️ Normalizamos el texto:
        # - quitamos espacios
        # - quitamos comas y $
        # - pasamos a mayúsculas
        clean_df = clean_df.withColumn(
            col,
            F.upper(
                F.regexp_replace(
                    F.regexp_replace(F.trim(F.col(col)), ",", ""),
                    "\\$", ""
                )
            )
        )

        # 2️ Convertimos valores especiales a NULL
        clean_df = clean_df.withColumn(
            col,
            F.when(
                (F.col(col).isNull()) |
                (F.col(col).isin("UNKNOWN", "?")),
                None
            )
            # 3️ Valores terminados en M → millones
            .when(
                F.col(col).endswith("M"),
                F.regexp_replace(F.col(col), "M", "").cast(DoubleType()) * 1e6
            )
            # 4️ Valores terminados en B → billones
            .when(
                F.col(col).endswith("B"),
                F.regexp_replace(F.col(col), "B", "").cast(DoubleType()) * 1e9
            )
            # 5️ Intento directo de conversión a número
            .otherwise(F.col(col).cast(DoubleType()))
        )

        # 6️ Calculamos la media de la columna
        mean_val = clean_df.select(F.mean(col)).first()[0]

        # 7️ Rellenamos los NULL con la media
        if mean_val is not None:
            clean_df = clean_df.na.fill({col: mean_val})
            print(f"Columna '{col}' limpiada y rellenada con media: {mean_val}")
        else:
            clean_df = clean_df.na.fill({col: 0.0})
            print(f"Columna '{col}' sin valores válidos → rellenada con 0.0")

print("\n===== 3. Limpieza de columnas categóricas =====")

# Columnas categóricas = todas menos las numéricas tratadas antes
categorical_cols = [
    c for c in clean_df.columns
    if c not in ["copies_sold_millions", "revenue_millions_usd"]
]

for col in categorical_cols:

    # 1️ Limpiamos espacios y normalizamos texto
    clean_df = clean_df.withColumn(
        col,
        F.trim(F.col(col))
    )

    # 2️ Reemplazamos valores problemáticos por NULL
    clean_df = clean_df.withColumn(
        col,
        F.when(
            (F.col(col).isNull()) |
            (F.col(col) == "") |
            (F.upper(F.col(col)).isin("?", "N/A", "UNKNOWN")),
            None
        ).otherwise(F.col(col))
    )

    # 3️ Rellenamos los NULL con 'Unknown'
    clean_df = clean_df.na.fill({col: "Unknown"})

print("Columnas categóricas limpiadas correctamente")


===== 2. Limpieza de columnas de ventas/ingresos =====
Columna 'copies_sold_millions' limpiada y rellenada con media: 10308561.989154695
Columna 'revenue_millions_usd' limpiada y rellenada con media: 507393576.9315999

===== 3. Limpieza de columnas categóricas =====
Columnas categóricas limpiadas correctamente


# 5. Creación de Dimensiones

Se crean las dimensiones usando distinct() y orderBy() en Spark. Se convierten a Pandas para asignar IDs y se recrean como DataFrames de Spark para los joins posteriores.

In [28]:
# Normalización de nombres de columnas

normalized_cols = [
    c.strip().lower().replace(" ", "_").replace("-", "_") for c in clean_df.columns
]
clean_df = clean_df.toDF(*normalized_cols)

clean_df.printSchema()


root
 |-- name: string (nullable = false)
 |-- genre: string (nullable = false)
 |-- cost: string (nullable = false)
 |-- platform: string (nullable = false)
 |-- popularity: string (nullable = false)
 |-- pegi: string (nullable = false)
 |-- year: string (nullable = false)
 |-- developer: string (nullable = false)
 |-- publisher: string (nullable = false)
 |-- region: string (nullable = false)
 |-- mode: string (nullable = false)
 |-- engine: string (nullable = false)
 |-- award: string (nullable = false)
 |-- dlc_support: string (nullable = false)
 |-- language: string (nullable = false)
 |-- metascore: string (nullable = false)
 |-- user_score: string (nullable = false)
 |-- reviews: string (nullable = false)
 |-- rating_source: string (nullable = false)
 |-- copies_sold_millions: double (nullable = false)
 |-- revenue_millions_usd: double (nullable = false)



# 6. Construcción de la Tabla de Hechos

Se construye la tabla de hechos mediante joins left entre el dataset y las dimensiones. Se seleccionan las columnas finales con IDs y métricas para completar el esquema en estrella.

In [29]:
from pyspark.sql import functions as F

cols = clean_df.columns
required_cols = ["name", "genre", "platform", "developer", "publisher", "year"]
for c in required_cols:
    if c not in cols:
        raise ValueError(f"Columna requerida no encontrada en el CSV: {c}")

print("Columnas disponibles después de normalización:")
print(cols)

print("\n===== Creación de dimensiones =====")

# Dimensión: dim_game
dim_game_pd = (
    clean_df.select("name", "genre")
    .distinct()
    .orderBy("name", "genre")
    .toPandas()
)
dim_game_pd["id_game"] = range(1, len(dim_game_pd) + 1)
dim_game = spark.createDataFrame(dim_game_pd)
print(f"Dimensión 'dim_game' creada: {len(dim_game_pd)} filas")

# Dimensión: dim_platform
dim_platform_pd = clean_df.select("platform").distinct().orderBy("platform").toPandas()
dim_platform_pd["id_platform"] = range(1, len(dim_platform_pd) + 1)
dim_platform = spark.createDataFrame(dim_platform_pd)
print(f"Dimensión 'dim_platform' creada: {len(dim_platform_pd)} filas")

# Dimensión: dim_developer
dim_developer_pd = clean_df.select("developer").distinct().orderBy("developer").toPandas()
dim_developer_pd["id_developer"] = range(1, len(dim_developer_pd) + 1)
dim_developer = spark.createDataFrame(dim_developer_pd)
print(f"Dimensión 'dim_developer' creada: {len(dim_developer_pd)} filas")

# Dimensión: dim_publisher
dim_publisher_pd = clean_df.select("publisher").distinct().orderBy("publisher").toPandas()
dim_publisher_pd["id_publisher"] = range(1, len(dim_publisher_pd) + 1)
dim_publisher = spark.createDataFrame(dim_publisher_pd)
print(f"Dimensión 'dim_publisher' creada: {len(dim_publisher_pd)} filas")

# Dimensión: dim_year
dim_year_pd = clean_df.select("year").distinct().orderBy("year").toPandas()
dim_year_pd["id_year"] = range(1, len(dim_year_pd) + 1)
dim_year = spark.createDataFrame(dim_year_pd)
print(f"Dimensión 'dim_year' creada: {len(dim_year_pd)} filas")

Columnas disponibles después de normalización:
['name', 'genre', 'cost', 'platform', 'popularity', 'pegi', 'year', 'developer', 'publisher', 'region', 'mode', 'engine', 'award', 'dlc_support', 'language', 'metascore', 'user_score', 'reviews', 'rating_source', 'copies_sold_millions', 'revenue_millions_usd']

===== Creación de dimensiones =====
Dimensión 'dim_game' creada: 408 filas
Dimensión 'dim_platform' creada: 9 filas
Dimensión 'dim_developer' creada: 13 filas
Dimensión 'dim_publisher' creada: 12 filas


[Stage 466:>                                                        (0 + 1) / 1]

Dimensión 'dim_year' creada: 42 filas


# 7. Carga en SQLite

Se carga el Data Warehouse en SQLite: se convierten los DataFrames de Spark a Pandas y se usan to_sql para insertar dimensiones y hechos en la base de datos.

In [30]:
# -------------------------------------------------------
# Construcción de la tabla de hechos (fact_sales)
# -------------------------------------------------------
fact = (
    clean_df
    .join(dim_game, on=["name", "genre"], how="left")
    .join(dim_platform, on=["platform"], how="left")
    .join(dim_developer, on=["developer"], how="left")
    .join(dim_publisher, on=["publisher"], how="left")
    .join(dim_year, on=["year"], how="left")
)

# Columnas métricas
value_cols = [c for c in ["copies_sold_millions", "revenue_millions_usd"] if c in fact.columns]
if not value_cols:
    raise ValueError("No se encontraron columnas métricas para construir la tabla de hechos.")

# Selección final de la tabla de hechos
fact_sales = fact.select(
    "id_game", "id_platform", "id_developer", "id_publisher", "id_year", *value_cols
)

print(f"\nTabla de hechos 'fact_sales' creada: {fact_sales.count()} filas")
fact_sales.show(5)



Tabla de hechos 'fact_sales' creada: 10000 filas
+-------+-----------+------------+------------+-------+--------------------+--------------------+
|id_game|id_platform|id_developer|id_publisher|id_year|copies_sold_millions|revenue_millions_usd|
+-------+-----------+------------+------------+-------+--------------------+--------------------+
|    200|          3|           5|          10|     23|           1500000.0|               282.3|
|    339|          4|          12|          10|     39|               34.55|               5.0E8|
|    373|          1|           8|           9|     33|1.0308561989154695E7|               1.0E9|
|     38|          7|           1|          10|     41|           1500000.0| 5.073935769315999E8|
|    371|          6|           9|           6|     34|               12.79| 5.073935769315999E8|
+-------+-----------+------------+------------+-------+--------------------+--------------------+
only showing top 5 rows


# 8. Consultas y Validación

Se ejecuta una consulta SQL de ejemplo para validar el Data Warehouse: se calcula el top 10 de géneros por ventas usando joins entre fact_sales y dim_game.

In [ ]:
# 9. Consultas y Validación

===== Carga de tablas en SQLite =====
Directorio warehouse: /app/warehouse
Base de datos: /app/warehouse/warehouse_pyspark.db

Cargando tablas en SQLite...


OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)

# 9. Resumen Final

Se resume el proceso ETL completado con PySpark: se confirma la carga exitosa, se listan las tablas creadas y se destaca el uso de Spark para procesamiento distribuido.

In [ ]:
# Ejemplo de consulta sobre el Data Warehouse generado por PySpark

print("===== Consulta: Top 10 géneros por ventas =====")

query = """
SELECT 
    g.genre, 
    SUM(f.copies_sold_millions) AS total_copies_sold,
    SUM(f.revenue_millions_usd) AS total_revenue
FROM fact_sales f
JOIN dim_game g ON f.id_game = g.id_game
GROUP BY g.genre
ORDER BY total_copies_sold DESC
LIMIT 10;
"""

print("Consulta SQL ejecutada:\n", query)

# Usar la misma conexión que creamos arriba
with engine.connect() as conn:
    top_genres_spark = pd.read_sql(query, conn)

print("\n===== Resultado: Top 10 géneros por ventas =====")
print(top_genres_spark)

print("\n===== Resumen del Data Warehouse =====")
print(f"Base de datos creada en: {db_path}")
print("\nTablas creadas:")
print("  - dim_game (dimensiones de juegos)")
print("  - dim_platform (dimensiones de plataformas)")
print("  - dim_developer (dimensiones de desarrolladores)")
print("  - dim_publisher (dimensiones de publishers)")
print("  - dim_year (dimensiones de años)")
print("  - fact_sales (tabla de hechos con ventas)")
print("\n✅ Proceso ETL con PySpark completado exitosamente")

===== Consulta: Top 10 géneros por ventas =====
Consulta SQL ejecutada:
 
SELECT 
    g.genre, 
    SUM(f.copies_sold_millions) AS total_copies_sold,
    SUM(f.revenue_millions_usd) AS total_revenue
FROM fact_sales f
JOIN dim_game g ON f.id_game = g.id_game
GROUP BY g.genre
ORDER BY total_copies_sold DESC
LIMIT 10;


===== Resultado: Top 10 géneros por ventas =====
       genre  total_copies_sold  total_revenue
0  Adventure       14194.671731  379461.549909
1     Puzzle       14144.021904  370930.434640
2          ?       14010.084514  338694.744293
3        RPG       14006.675556  368915.969530
4     Racing       13935.575817  336646.674514
5       RPG        13930.538948  370828.224356
6     Sports       13580.824427  362876.689467
7    Unknown       13575.054601  373894.664546
8     action       13468.101123  336632.654577
9      Indie       13277.771731  330911.439309


# 10. Resumen Final

Se muestra el resultado de la consulta y se proporciona un resumen completo del Data Warehouse generado, incluyendo estadísticas y confirmación de objetivos cumplidos.